## Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.



Коллеги, предоставившие данные по исследованиям, описали для нас каждый вид переменной из таблицы. 

- children — количество детей в семье 
- days_employed — общий трудовой стаж в днях
- dob_years — возраст клиента в годах
- education — уровень образования клиента
- education_id — идентификатор уровня образования
- family_status — семейное положение
- family_status_id — идентификатор семейного положения
- gender — пол клиента
- income_type — тип занятости
- debt — имел ли задолженность по возврату кредитов
- total_income — ежемесячный доход
- purpose — цель получения кредита


## План работы 

Для себя я разметил следующий план работы над данным проектом:

    1) Необходимо ознакомиться поближе с данными: просмотреть, есть ли пропуски в столбцах, предположить, почему они появились. Обязательно проверить на соответствие типов данных: возможно, мы будем работать с типом datetime, который будет представлен типом str. 
    2) После ознаекомления с таблицей необходимо внести изменения: убрать пропуски, привести к корректным типам данных, а также просмотреть на наличие дубликатов. 
    3) Когда таблица будет готова для дальнейшей работы, необходимо будет посмотреть на поставленные мне вопросы - из них мне станет понятно, какие возможные виды категоризации можно будет применить. Необходимо не забыть про лемматизацию, так как у нас есть такой вид переменной, как purpose.
    4) Когда у меня будут категории вместе с завершенной лемматизацией, будет необходимо совершить проверку выдвинутых заказчиками гипотез.
    5) После окончания работы необходимо будет сделать сооответствующие выводы по проекту.

### Шаг 1. Откройте файл с данными и изучите общую информацию. 

In [1]:
import pandas as pd
from IPython.display import display
df = pd.read_csv('/datasets/data.csv') 

df.info()

display(df.head(10))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


### Вывод

1) Таблица явно перегружена: слишком много столбцов, читать сложно. Необходимо сократить её до более-менее читабельного вида. К примеру, избавиться от столбцов, описывающих семейное положение (словесно), а также образование. Для данных столбцов предусмотрели id. 


2) В таблице присутствуют пропуски значений. Необходимо определиться, что мы с ними будем делать. 



### Шаг 2. Предобработка данных

### Обработка пропусков

In [2]:
# посмотрел в канале slack'a более элегантный способ создать словарь медиан от нашего преподователя
total_income_dict = df.groupby('income_type')['total_income'].median().to_dict() 

def fillna_days(row):
    if pd.isna(row['days_employed']):
        return df['days_employed'].median()
    return row['days_employed']

def fillna_income_type(row):
    if pd.isna(row['total_income']):
        return total_income_dict[row['income_type']]
    return row['total_income']
           
           
df['days_employed'] = df.apply(fillna_days, axis=1)
df['total_income'] = df.apply(fillna_income_type, axis=1)


df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


### Вывод

При помощи метода DataFrame.info() обнаружили пропуски в количественных переменных: days_employed и total_income. 

Общее количество пропусков - 21 525-19 351=2 174, что составляет 10% от общего числа значений. Это много. 

Столбец days_employed дальше нигде не используется, заполним её с помощью простой функции среднего арифметического .mean().
Столбец total_income может исказить нам результат, дальше она будет использована. Поэтому был создан словарь с медианными значениями дохода за месяц (при помощи методов median() и to_dict()), при помощи него и были заполнены пропуски.



### Замена типа данных

In [3]:

#Преобразуем столбец значений float в столбец значений int
#берём метод astype(), так как метод to_numeric() предназначен для переведения из типа str в тип float
df['days_employed'] = df['days_employed'].astype('int')

df.info() # смотрим на изменения

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null int64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null float64
purpose             21525 non-null object
dtypes: float64(1), int64(6), object(5)
memory usage: 2.0+ MB


### Вывод

Столбец days_employed указывает на дни, которые должны измеряться в целых числах (integer). При этом все значения указаны в параметре float. Исправим этот казус при помощи метода astype().

<div class="alert alert-success">
<h2> Комментарий от ревьюера</h2>

Хорошо. Так же можно округлить до целого доход клиентов.

</div>


### Обработка дубликатов

In [4]:
def dict_create(data, selected_column):
    dictionary = data
    dictionary[selected_column] = data[selected_column].str.lower()
    dictionary = dictionary.drop_duplicates().reset_index(drop=True)
    return dictionary


# создаем словарь семейных статусов
family_dict = dict_create(df.loc[:, ('family_status', 'family_status_id')], 'family_status')
                        
# создаем словарь по образованию
education_dict = dict_create(df.loc[: ,('education', 'education_id')], 'education')

# создаем компактный DataFrame
compact_df = df.loc[:, ('children','days_employed', 'dob_years', 'education_id', 'family_status_id', 'gender', 
               'income_type', 'debt', 'total_income', 'purpose')]

print('Количество дублирующихся строк после всех преобразований: {}'.format(compact_df.duplicated().sum()))

# избавляемся от дубликатов с переписыванием индексов
compact_df = compact_df.drop_duplicates().reset_index(drop=True)

# выведем таблицу, чтобы посмотреть на неё (она выглядит прекрасно!)
display(compact_df.head(5))



Количество дублирующихся строк после всех преобразований: 71


,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437,42,0,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024,36,1,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623,33,1,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124,32,1,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266,53,1,1,F,пенсионер,0,158616.077870,сыграть свадьбу


### Вывод

Проверка на полное совпадение строк дала 54 дубликата. Избавимся от них при помощи метода drop_duplicate(), а также reset_index(). Дубликаты могли возникнуть из-за технических сбоев в выгрузке.


Теперь необходимо посмотреть, что у нас со столбцами.
У нас есть множество дублирующихся значений в следующих столбцах: family_status, education, income_type.

Как итог работы были созданы соответствующие словари. Из датафрейма убрали family_status, education, а также преобразовали income_type для простоты работы с таблицей.

### Лемматизация

In [5]:
from pymystem3 import Mystem # импортируем из библиотеки pymystem3 класс Mystem
from collections import Counter # импортируем из модуля контейнер Counter

m = Mystem() # создаем переменную типа Mystem

# создаем столбец списков лемматизированных слов и заполняем его на основе лемматизации столбца 'purpose'
compact_df['purpose_lemmatized'] = compact_df['purpose'].apply(m.lemmatize) 

#собирает все лемматезированные элементы в одну строку и считаем все совпадения Counter'ом
print(Counter(compact_df['purpose_lemmatized'].sum())) 

Counter({' ': 33570, '\n': 21454, 'недвижимость': 6351, 'покупка': 5897, 'жилье': 4460, 'автомобиль': 4306, 'образование': 4013, 'с': 2918, 'операция': 2604, 'свадьба': 2324, 'свой': 2230, 'на': 2222, 'строительство': 1878, 'высокий': 1374, 'получение': 1314, 'коммерческий': 1311, 'для': 1289, 'жилой': 1230, 'сделка': 941, 'дополнительный': 906, 'заниматься': 904, 'проведение': 768, 'сыграть': 765, 'сдача': 651, 'семья': 638, 'собственный': 635, 'со': 627, 'ремонт': 607, 'подержанный': 486, 'подержать': 478, 'приобретение': 461, 'профильный': 436})


### Вывод

Лемматизация показала, что основными категориями, по которым берут кредиты, являются покупка недвижимости/жилья/строительству, покупка автомобиля, кредит с целью получения образования, а также кредит на свадьбу.

### Категоризация данных

In [11]:
#при помощи лемматизации определили основные причины, почему берётся кредит
#поставил категории сверху, так как они есть глобальные переменные
auto = ['автомобиль']
education = ['образование']
housing = ['недвижимость', 'жилье', 'строительство']
wedding = ['свадьба'] 


#собрали вместе функции, при помощи которых осуществляется категоризация;
#функции имеют одинаковое ключевое слово в наименовании для простоты в поиске и идентификации
def categoryzation_kiddos(kids): 
    if kids == 0:
        return 'детей нет'
    if kids > 0 and kids <= 2:
        return 'от 1 до 2 детей'
    if kids > 2:
        return 'многодетный родитель'

print(compact_df['total_income'].describe(percentiles = [.1,.9])) 
#посмотрел, что к чему по распрелению 'total_income'
#взял значения и использовал
    
def categoryzation_income(money):
    if money <= 98538.27:
        return '0 - 98.538,27'
    if money > 98538.27 and money <= 132135.1:
        return '98.538,27 - 132.135,1'
    if money > 132135.1 and money <= 161335.7:
        return '132.135,1 - 161.335,7'
    if money > 161335.7 and money <= 214618.0:
        return '161.335,7 - 214.618,0'
    if money > 214618.0:
        return '214.618,0 и более'
    
def categoryzation_lemms(lemms):
    for x in lemms:
        if x in auto:
            return 'Покупка авто'
        if x in education:
            return 'Образование'
        if x in housing:
            return 'Операции с недвижимостью'
        if x in wedding:
            return 'Кредит на свадьбу'
    return 'другое'

compact_df['child_cat'] = compact_df['children'].apply(categoryzation_kiddos)
compact_df['income_cat'] = compact_df['total_income'].apply(categoryzation_income)
compact_df['lemms_cat'] = compact_df['purpose_lemmatized'].apply(categoryzation_lemms)


print(compact_df.head(10))


count    2.145400e+04
mean     1.653201e+05
std      9.818730e+04
min      2.066726e+04
10%      7.872156e+04
50%      1.425944e+05
90%      2.698257e+05
max      2.265604e+06
Name: total_income, dtype: float64
   children  days_employed  dob_years  education_id  family_status_id gender  \
0         1          -8437         42             0                 0      F   
1         1          -4024         36             1                 0      F   
2         0          -5623         33             1                 0      M   
3         3          -4124         32             1                 0      M   
4         0         340266         53             1                 1      F   
5         0           -926         27             0                 1      M   
6         0          -2879         43             0                 0      F   
7         0           -152         50             1                 0      M   
8         2          -6929         35             0                 1

### Вывод

Сгруппируем наших клиентов на группы по многодетности: детей нет, от 1 до 2 детей, многодетный родитель.

Добавим категорию, которая укажет на уровень доходов лиц, занимающих средства. Укажем шаг в 50 тыс. руб.

При помощи лемматизации выясним причины, по которым берутся кредиты. Благодаря успешному подбору категорий не появилось категории "другое", показывающей, что мы охватили не все причины. 

### Шаг 3. Ответьте на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [7]:
pivot_data_child = compact_df.pivot_table(index='child_cat', values='debt', aggfunc=['count','sum', 'mean'])

display(pivot_data_child)

,count,sum,mean
,debt,debt,debt
child_cat,,,
детей нет,14091,1063,0.075438
многодетный родитель,456,39,0.085526
от 1 до 2 детей,6860,638,0.093003


### Вывод

На основании данных, полученных в результате анализа, получаем следующее:

Действительно, платежеспособность наиболее высока у категории лиц "бездетные", при этом наибольший процент невозврата кредита в срок наблюдается у категории лиц с от 1 до 2 детей. Это может быть вызвано наличием государственной поддержки для многодетных семей с рядом льгот в виде дополнительного питания, субсидирования жилья и так далее.

Вывод - зависимость есть.

- Есть ли зависимость между семейным положением и возвратом кредита в срок?


In [8]:
pivot_data_fam = compact_df.pivot_table(index='family_status_id', values='debt', aggfunc=['count','sum', 'mean'])


pivot_data_fam.columns = ['count', 'sum', 'mean'] 

pivot_data_fam = family_dict.merge(pivot_data_fam, on='family_status_id', how='right')

display(pivot_data_fam)


,family_status,family_status_id,count,sum,mean
0,женат / замужем,0,12339,931,0.075452
1,гражданский брак,1,4151,388,0.093471
2,вдовец / вдова,2,959,63,0.065693
3,в разводе,3,1195,85,0.071130
4,не женат / не замужем,4,2810,274,0.097509


### Вывод

Наиболее кредитоспособными показывают себя лица, являющиеся вдовцами или вдовами. При этом, категория незамужних и неженатых показывает худший результат по уровню возврата кредита в срок.

Это можно связать с тем, что неженатые/незамужние граждане не участвовали в совместной жизни и не устраивали совместный бюджет - у них могут быть сложности с планированием, так как они одиноки и не обременены связями с семьей.

Вывод - зависимость есть.

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [9]:
pivot_data_inc = compact_df.pivot_table(index='income_cat', values='debt', aggfunc=['count','sum', 'mean'])


display(pivot_data_inc)


,count,sum,mean
,debt,debt,debt
income_cat,,,
"0 - 98.538,27",4291,344,0.080168
"132.135,1 - 161.335,7",4290,375,0.087413
"161.335,7 - 214.618,0",4291,361,0.084130
"214.618,0 и более",4291,300,0.069914
"98.538,27 - 132.135,1",4291,361,0.084130


### Вывод

Четкой зависимости по проценту возвратов кредита в срок не прослеживается. Процент возврата не в срок для групп от 0 тыс. руб. до 215 тыс. руб. колеблется в пределах 1 процентного пункта.

При этом, у лиц, чей месячных доход превышает 214 тыс. руб., процент дохода уменьшается на более чем 1 процентный пункт. Можно сделать вывод, что наиболее богатая часть граждан лучше всего выплачивает кредиты. Остальные же - примерно одинаково.

- Как разные цели кредита влияют на его возврат в срок?

In [10]:
pivot_data_lemms = compact_df.pivot_table(index='lemms_cat', values='debt', aggfunc=['count','sum', 'mean'])

display(pivot_data_lemms)

,count,sum,mean
,debt,debt,debt
lemms_cat,,,
Кредит на свадьбу,2324,186,0.080034
Образование,4013,370,0.092200
Операции с недвижимостью,10811,782,0.072334
Покупка авто,4306,403,0.093590


### Вывод

На основании анализа видим, что покупка авто и образование имеют наибольший процент невыплат кредита в срок.

Кредит на свадьбу и операции с недвижимостью имеют меньший процент невыплат в срок. Это может быть связано с тем, что данные кредиты чаще всего берутся и закрываются скорее семьёй, нежели одним человеком.  

### Шаг 4. Общий вывод

#### Результаты исследования


По результатам исследования стало понятно, что семейное полоение и количество детей клиента оказывает влияние на факт погашения кредита в срок. 

Так, наименьший процент погашения кредита не в срок имеет категория лиц "без детей" - **7,54%**. У категории лиц, имеющих от 1 до 2 детей, имеет **9,3%**.

По семейному положению наблюдается следующая зависимость. Категории лиц без, находящихся в отношениях, перешедших хотя бы в категорию "гражданских брак", имеет наибольший процент невозврата в срок - 	**9,75%**. Во всех остальных категориях наблюдается уменьшение процента невозврата: наиболее репрезентативной выглядит процент невозвратов кредита женатых / замужних кредиторов **7,54%**.



#### Идеальный кредитор

Группировка по различным параметрам показала, что наименьший процент невыплат будет наблюдаться у лиц, состоящих в отношениях (в гражданском браке или женатые/замужние) без детей, зарабатывающих в месяц не менее 200 тыс. руб., берущих кредит на операции с недвижимостью. 


#### Проблемы с выгрузкой

При этом, при работе с данной выгрузкой возникло несколько вопросов:
1) Почему такие странные значения в части количества рабочих дней? 
2) Почему в некоторых случаях возраст указан меньше 18 лет?

Данные вопросы лучше всего направить авторам выгрузки, чтобы они могли улучшить систему для более качественной и быстрой работы с данными, а также возможностей автоматизации.